In [1]:
import enum
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
from urllib.request import urlretrieve

import libsbml
from process_bigraph import pp 

from test_suite.test_data_model import TestCompositionResults

In [2]:
test_archive_fp = '../test_suite/examples/sbml-core/Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML.omex'
os.path.exists(test_archive_fp)

In [3]:
from dataclasses import dataclass


@dataclass(frozen=True)
class A:
    x: float
    y: float 
    

a1 = A(x=0.5, y=0.5 / 2)

a1.x

In [4]:
a1.x = 0.3


In [ ]:
@dataclass
class B:
    x: float
    y: float

In [ ]:
b1 = B(x=0.5, y=0.5)

b1.x

In [ ]:
b1.x = 0.3

b1.x